In [1]:
from IsingLib import Model, Hamil, DaSampler
from pyqubo import Array, Sum
import numpy as np
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite


np.set_printoptions(threshold=30)
np.set_printoptions(edgeitems=20)
np.set_printoptions(linewidth=200)

In [2]:
city = 5
vartype = 'BINARY'
x = Array.create('x', (city, city), vartype)
dis_li = [[0, 10, 8, 8, 4], [10, 0, 6, 11, 7], [8, 6, 0, 14, 8], [9, 11, 14, 0, 6], [4, 7, 8, 6, 0]]
distance = np.zeros((city**2, city**2))
for i in range(city):
    point = (i + 1) % city
    distance[i*city: (i+1)*city, point*city: (point+1)*city] = dis_li
distance = distance + distance.T
time_const = Sum(0, city, lambda i: (Sum(0, city, lambda j: x[i, j]) - 1) ** 2)
city_const = Sum(0, city, lambda j: (Sum(0, city, lambda i: x[i, j]) - 1) ** 2)

In [3]:
const_model = Model.create(vartype, time_const+city_const)
aim_model = Model(vartype, distance, symbol='x', shape=(city, city))

In [4]:
h = Hamil(aim=aim_model, const=const_model)

In [5]:
print(h)

H = {'aim': <IsingLib.model.Model object at 0x10be20910>, 'const': <IsingLib.model.Model object at 0x10be28690>}


In [6]:
model = h.compile(const=10)
print(model.const)

100.0


In [7]:
ans = model.solve('sa', iteration=2)
print(ans)

In [8]:
print(ans.states)
print(ans.enegries)
print(ans.frequency)

[{'x[0][0]': 1, 'x[0][1]': 0, 'x[0][2]': 0, 'x[0][3]': 0, 'x[0][4]': 0, 'x[1][0]': 0, 'x[1][1]': 0, 'x[1][2]': 0, 'x[1][3]': 1, 'x[1][4]': 0, 'x[2][0]': 0, 'x[2][1]': 0, 'x[2][2]': 0, 'x[2][3]': 0, 'x[2][4]': 1, 'x[3][0]': 0, 'x[3][1]': 1, 'x[3][2]': 0, 'x[3][3]': 0, 'x[3][4]': 0, 'x[4][0]': 0, 'x[4][1]': 0, 'x[4][2]': 1, 'x[4][3]': 0, 'x[4][4]': 0}]
[35.0]
[2]


In [19]:
for state in ans.states:
    print(h.check_constraints(state, 'const'))

0.0


In [10]:
# import itertools

# for v in itertools.permutations(range(city)):
#     d = 0
#     for i in range(city):
#         d += dis_li[v[i-1]][v[i]]
#     print(d, v)

In [10]:
model = h.compile(const=10)
da_model = model.to_da()
print(da_model)

da = DaSampler(**da_model, api_key='../../../da_key_2.json', version=2)
print(da)

{'model_type': 'qubo', 'matrix': array([[-20.,  20.,  20.,  20.,  20.,  20.,  10.,   8.,   8.,   4.,  20.,   0.,   0.,   0.,   0.,  20.,   0.,   0.,   0.,   0.,  20.,  10.,   8.,   9.,   4.],
       [  0., -20.,  20.,  20.,  20.,  10.,  20.,   6.,  11.,   7.,   0.,  20.,   0.,   0.,   0.,   0.,  20.,   0.,   0.,   0.,  10.,  20.,   6.,  11.,   7.],
       [  0.,   0., -20.,  20.,  20.,   8.,   6.,  20.,  14.,   8.,   0.,   0.,  20.,   0.,   0.,   0.,   0.,  20.,   0.,   0.,   8.,   6.,  20.,  14.,   8.],
       [  0.,   0.,   0., -20.,  20.,   9.,  11.,  14.,  20.,   6.,   0.,   0.,   0.,  20.,   0.,   0.,   0.,   0.,  20.,   0.,   8.,  11.,  14.,  20.,   6.],
       [  0.,   0.,   0.,   0., -20.,   4.,   7.,   8.,   6.,  20.,   0.,   0.,   0.,   0.,  20.,   0.,   0.,   0.,   0.,  20.,   4.,   7.,   8.,   6.,  20.],
       [  0.,   0.,   0.,   0.,   0., -20.,  20.,  20.,  20.,  20.,  20.,  10.,   8.,   8.,   4.,  20.,   0.,   0.,   0.,   0.,  20.,   0.,   0.,   0.,   0.],
       [  0.,

In [11]:
job_id = da.sample('da')
print(job_id)
response = da.get_job(job_id)

PV83946-191981020186830


In [32]:
print(response.states)
print(response.frequency)
print(response.enegries)

[{'x[0][0]': 0, 'x[0][1]': 0, 'x[2][0]': 0, 'x[2][1]': 1, 'x[2][2]': 0, 'x[2][3]': 0, 'x[2][4]': 0, 'x[3][0]': 0, 'x[3][1]': 0, 'x[3][2]': 1, 'x[3][3]': 0, 'x[3][4]': 0, 'x[0][2]': 0, 'x[4][0]': 1, 'x[4][1]': 0, 'x[4][2]': 0, 'x[4][3]': 0, 'x[4][4]': 0, 'x[0][3]': 0, 'x[0][4]': 1, 'x[1][0]': 0, 'x[1][1]': 0, 'x[1][2]': 0, 'x[1][3]': 1, 'x[1][4]': 0}, {'x[0][0]': 0, 'x[0][1]': 0, 'x[2][0]': 0, 'x[2][1]': 0, 'x[2][2]': 1, 'x[2][3]': 0, 'x[2][4]': 0, 'x[3][0]': 1, 'x[3][1]': 0, 'x[3][2]': 0, 'x[3][3]': 0, 'x[3][4]': 0, 'x[0][2]': 0, 'x[4][0]': 0, 'x[4][1]': 0, 'x[4][2]': 0, 'x[4][3]': 1, 'x[4][4]': 0, 'x[0][3]': 0, 'x[0][4]': 1, 'x[1][0]': 0, 'x[1][1]': 1, 'x[1][2]': 0, 'x[1][3]': 0, 'x[1][4]': 0}, {'x[0][0]': 0, 'x[0][1]': 0, 'x[2][0]': 0, 'x[2][1]': 0, 'x[2][2]': 1, 'x[2][3]': 0, 'x[2][4]': 0, 'x[3][0]': 0, 'x[3][1]': 1, 'x[3][2]': 0, 'x[3][3]': 0, 'x[3][4]': 0, 'x[0][2]': 0, 'x[4][0]': 0, 'x[4][1]': 0, 'x[4][2]': 0, 'x[4][3]': 1, 'x[4][4]': 0, 'x[0][3]': 0, 'x[0][4]': 1, 'x[1][0]': 1, 

In [34]:
for state in response.states:
    print(h.check_constraints(state, 'const'))

TypeError: unsupported operand type(s) for *: 'float' and 'dict'

In [8]:
model_dwave = h.compile(const=10)
bqm = model_dwave.to_bqm()
print(bqm)

sampler = EmbeddingComposite(DWaveSampler())

response = sampler.sample(bqm)
print(response.record[0][0].reshape(city, city))

BinaryQuadraticModel({'x[0][0]': -20.0, 'x[0][1]': -20.0, 'x[0][2]': -20.0, 'x[0][3]': -20.0, 'x[0][4]': -20.0, 'x[1][0]': -20.0, 'x[1][1]': -20.0, 'x[1][2]': -20.0, 'x[1][3]': -20.0, 'x[1][4]': -20.0, 'x[2][0]': -20.0, 'x[3][0]': -20.0, 'x[4][0]': -20.0, 'x[4][1]': -20.0, 'x[4][2]': -20.0, 'x[4][3]': -20.0, 'x[4][4]': -20.0, 'x[2][1]': -20.0, 'x[3][1]': -20.0, 'x[2][2]': -20.0, 'x[3][2]': -20.0, 'x[2][3]': -20.0, 'x[3][3]': -20.0, 'x[2][4]': -20.0, 'x[3][4]': -20.0}, {('x[0][0]', 'x[0][1]'): 20.0, ('x[0][0]', 'x[0][2]'): 20.0, ('x[0][0]', 'x[0][3]'): 20.0, ('x[0][0]', 'x[0][4]'): 20.0, ('x[0][0]', 'x[1][0]'): 20.0, ('x[0][0]', 'x[1][1]'): 10.0, ('x[0][0]', 'x[1][2]'): 8.0, ('x[0][0]', 'x[1][3]'): 8.0, ('x[0][0]', 'x[1][4]'): 4.0, ('x[0][0]', 'x[2][0]'): 20.0, ('x[0][0]', 'x[3][0]'): 20.0, ('x[0][0]', 'x[4][0]'): 20.0, ('x[0][0]', 'x[4][1]'): 10.0, ('x[0][0]', 'x[4][2]'): 8.0, ('x[0][0]', 'x[4][3]'): 9.0, ('x[0][0]', 'x[4][4]'): 4.0, ('x[0][1]', 'x[0][2]'): 20.0, ('x[0][1]', 'x[0][3]')

In [9]:
response

SampleSet(rec.array([([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 72., 1, 0.84)],
          dtype=[('sample', 'i1', (25,)), ('energy', '<f8'), ('num_occurrences', '<i8'), ('chain_break_fraction', '<f8')]), ['x[0][0]', 'x[0][1]', 'x[0][2]', 'x[0][3]', 'x[0][4]', 'x[1][0]', 'x[1][1]', 'x[1][2]', 'x[1][3]', 'x[1][4]', 'x[2][0]', 'x[2][1]', 'x[2][2]', 'x[2][3]', 'x[2][4]', 'x[3][0]', 'x[3][1]', 'x[3][2]', 'x[3][3]', 'x[3][4]', 'x[4][0]', 'x[4][1]', 'x[4][2]', 'x[4][3]', 'x[4][4]'], {'timing': {'qpu_sampling_time': 315, 'qpu_anneal_time_per_sample': 20, 'qpu_readout_time_per_sample': 274, 'qpu_access_time': 10356, 'qpu_access_overhead_time': 3170, 'qpu_programming_time': 10041, 'qpu_delay_time_per_sample': 21, 'total_post_processing_time': 496, 'post_processing_overhead_time': 496, 'total_real_time': 10356, 'run_time_chip': 315, 'anneal_time_per_run': 20, 'readout_time_per_run': 274}}, 'BINARY')

In [10]:
model_dwave.energy([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

72.0